# Tiny-ImageNet 图像分类实验

本实验基于Scoring Criteria 3，在tiny-imagenet数据集上实验5个改进因子来提升图像分类性能。

## 实验设计

### 数据集特点
- **tiny-imagenet**: 110,000张图像，64x64像素，200个类别
- 比CIFAR-10更复杂，需要更强的模型和训练策略

### 5个改进因子（与CIFAR-10相同）
1. **残差连接机制 (Residual Connections)** - 解决梯度消失问题
2. **更深/更宽的网络架构** - 增加模型容量
3. **更好的优化器和学习率调度** - 提升训练效果
4. **高级数据增强** - MixUp, CutMix, AutoAugment
5. **注意力机制** - SE Block, Self-Attention

### 实验流程
1. 数据集准备和加载
2. 基线模型训练和评估
3. 逐个添加改进因子，记录性能变化
4. 分析每个因子的贡献
5. 组合最佳因子进行最终实验


In [ ]:
# 导入必要的库
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import time
import os

# 导入自定义模块
from models import get_model
from dataset_utils import get_dataset_loaders
from training_utils import (
    train_model, get_optimizer, get_scheduler, 
    get_optimizer_config, get_scheduler_config
)
from data_augmentation import get_transforms

# 设置随机种子
torch.manual_seed(42)
np.random.seed(42)

# 设备配置
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"使用设备: {device}")


## 实验配置


In [ ]:
# 实验配置
config = {
    'dataset': 'tiny_imagenet',
    'batch_size': 32,  # 由于tiny-imagenet图像更大，减小batch_size
    'num_workers': 2,
    'num_epochs': 80,  # tiny-imagenet需要更多epoch
    'print_every': 200,
}

# 模型配置
models_to_test = [
    'baseline',           # 基线模型
    'residual',          # 改进因子(a): 残差连接
    'deeper_wider',      # 改进因子(b): 更深更宽网络
    'se_attention',      # 改进因子(e): SE注意力
    'self_attention',    # 改进因子(e): 自注意力
]

# 优化器配置 - 改进因子(c)
optimizer_configs = {
    'Adam': {'optimizer': 'Adam', 'scheduler': None},
    'AdamW': {'optimizer': 'AdamW', 'scheduler': 'cosine'},
    'SGD': {'optimizer': 'SGD', 'scheduler': 'step'},
}

# 数据增强配置 - 改进因子(d)
augmentation_configs = {
    'basic': 'basic',
    'autoaugment': 'autoaugment',
    'mixup': 'mixup',
    'cutmix': 'cutmix',
}

print("实验配置完成")


## 1. 数据集准备和加载


In [ ]:
# 加载tiny-imagenet数据集
print("正在准备tiny-imagenet数据集...")
train_loader, test_loader, num_classes = get_dataset_loaders(
    config['dataset'], 
    batch_size=config['batch_size'],
    num_workers=config['num_workers'],
    augmentation_type='basic'
)

print(f"数据集: {config['dataset']}")
print(f"类别数: {num_classes}")
print(f"训练集大小: {len(train_loader.dataset)}")
print(f"测试集大小: {len(test_loader.dataset)}")
print(f"训练批次数: {len(train_loader)}")
print(f"测试批次数: {len(test_loader)}")


## 2. 基线模型实验


In [ ]:
# 基线模型实验
print("=== 基线模型实验 ===")

# 创建基线模型
baseline_model = get_model('baseline', num_classes).to(device)

# 配置优化器
optimizer_config = get_optimizer_config('Adam')
optimizer = get_optimizer(baseline_model, 'Adam', **optimizer_config)

# 损失函数
criterion = nn.CrossEntropyLoss()

# 训练模型
baseline_history = train_model(
    baseline_model, train_loader, test_loader,
    config['num_epochs'], optimizer, criterion, device,
    print_every=config['print_every']
)

print(f"基线模型最终测试准确率: {max(baseline_history['test_acc']):.2f}%")


## 3. 改进因子实验

### 3.1 残差连接机制 (改进因子a)


In [ ]:
print("=== 改进因子(a): 残差连接机制 ===")

# 创建带残差连接的模型
residual_model = get_model('residual', num_classes).to(device)

# 使用相同的优化器配置
optimizer = get_optimizer(residual_model, 'Adam', **optimizer_config)

# 训练模型
residual_history = train_model(
    residual_model, train_loader, test_loader,
    config['num_epochs'], optimizer, criterion, device,
    print_every=config['print_every']
)

print(f"残差模型最终测试准确率: {max(residual_history['test_acc']):.2f}%")
print(f"相比基线提升: {max(residual_history['test_acc']) - max(baseline_history['test_acc']):.2f}%")


### 3.2 更深更宽的网络架构 (改进因子b)


In [ ]:
print("=== 改进因子(b): 更深更宽的网络架构 ===")

# 创建更深更宽的模型
deeper_model = get_model('deeper_wider', num_classes).to(device)

# 使用相同的优化器配置
optimizer = get_optimizer(deeper_model, 'Adam', **optimizer_config)

# 训练模型
deeper_history = train_model(
    deeper_model, train_loader, test_loader,
    config['num_epochs'], optimizer, criterion, device,
    print_every=config['print_every']
)

print(f"更深更宽模型最终测试准确率: {max(deeper_history['test_acc']):.2f}%")
print(f"相比基线提升: {max(deeper_history['test_acc']) - max(baseline_history['test_acc']):.2f}%")


### 3.3 更好的优化器和学习率调度 (改进因子c)


In [ ]:
print("=== 改进因子(c): 更好的优化器和学习率调度 ===")

# 测试不同的优化器配置
optimizer_results = {}

for opt_name, opt_config in optimizer_configs.items():
    print(f"\n测试优化器: {opt_name}")
    
    # 创建新模型
    model = get_model('baseline', num_classes).to(device)
    
    # 配置优化器
    opt_config_dict = get_optimizer_config(opt_config['optimizer'])
    optimizer = get_optimizer(model, opt_config['optimizer'], **opt_config_dict)
    
    # 配置学习率调度器
    scheduler = None
    if opt_config['scheduler']:
        scheduler_config = get_scheduler_config(opt_config['scheduler'], config['num_epochs'])
        scheduler = get_scheduler(optimizer, opt_config['scheduler'], **scheduler_config)
    
    # 训练模型
    history = train_model(
        model, train_loader, test_loader,
        config['num_epochs'], optimizer, criterion, device,
        scheduler=scheduler, print_every=config['print_every']
    )
    
    best_acc = max(history['test_acc'])
    optimizer_results[opt_name] = best_acc
    print(f"{opt_name} 最佳测试准确率: {best_acc:.2f}%")

# 找到最佳优化器
best_optimizer = max(optimizer_results, key=optimizer_results.get)
print(f"\n最佳优化器: {best_optimizer} (准确率: {optimizer_results[best_optimizer]:.2f}%)")


### 3.4 高级数据增强 (改进因子d)


In [ ]:
print("=== 改进因子(d): 高级数据增强 ===")

# 测试不同的数据增强方法
augmentation_results = {}

for aug_name, aug_type in augmentation_configs.items():
    print(f"\n测试数据增强: {aug_name}")
    
    # 重新加载数据（使用不同的数据增强）
    train_loader_aug, test_loader_aug, _ = get_dataset_loaders(
        config['dataset'], 
        batch_size=config['batch_size'],
        num_workers=config['num_workers'],
        augmentation_type=aug_type
    )
    
    # 创建模型
    model = get_model('baseline', num_classes).to(device)
    
    # 使用最佳优化器配置
    opt_config_dict = get_optimizer_config(best_optimizer)
    optimizer = get_optimizer(model, best_optimizer, **opt_config_dict)
    
    # 训练模型
    history = train_model(
        model, train_loader_aug, test_loader_aug,
        config['num_epochs'], optimizer, criterion, device,
        augmentation=aug_type if aug_type in ['mixup', 'cutmix'] else None,
        print_every=config['print_every']
    )
    
    best_acc = max(history['test_acc'])
    augmentation_results[aug_name] = best_acc
    print(f"{aug_name} 最佳测试准确率: {best_acc:.2f}%")

# 找到最佳数据增强方法
best_augmentation = max(augmentation_results, key=augmentation_results.get)
print(f"\n最佳数据增强: {best_augmentation} (准确率: {augmentation_results[best_augmentation]:.2f}%)")


### 3.5 注意力机制 (改进因子e)


In [ ]:
print("=== 改进因子(e): 注意力机制 ===")

# 测试SE注意力和自注意力
attention_results = {}

for attention_type in ['se_attention', 'self_attention']:
    print(f"\n测试注意力机制: {attention_type}")
    
    # 创建带注意力的模型
    model = get_model(attention_type, num_classes).to(device)
    
    # 使用最佳优化器配置
    opt_config_dict = get_optimizer_config(best_optimizer)
    optimizer = get_optimizer(model, best_optimizer, **opt_config_dict)
    
    # 使用最佳数据增强
    train_loader_aug, test_loader_aug, _ = get_dataset_loaders(
        config['dataset'], 
        batch_size=config['batch_size'],
        num_workers=config['num_workers'],
        augmentation_type=best_augmentation
    )
    
    # 训练模型
    history = train_model(
        model, train_loader_aug, test_loader_aug,
        config['num_epochs'], optimizer, criterion, device,
        augmentation=best_augmentation if best_augmentation in ['mixup', 'cutmix'] else None,
        print_every=config['print_every']
    )
    
    best_acc = max(history['test_acc'])
    attention_results[attention_type] = best_acc
    print(f"{attention_type} 最佳测试准确率: {best_acc:.2f}%")


## 4. 实验结果分析和可视化


In [ ]:
# 整理所有实验结果
all_results = {
    'baseline': max(baseline_history['test_acc']),
    'residual': max(residual_history['test_acc']),
    'deeper_wider': max(deeper_history['test_acc']),
    **optimizer_results,
    **augmentation_results,
    **attention_results
}

# 创建结果表格
results_df = pd.DataFrame(list(all_results.items()), columns=['Method', 'Accuracy'])
results_df = results_df.sort_values('Accuracy', ascending=False)

print("=== Tiny-ImageNet 实验结果汇总 ===")
print(results_df.to_string(index=False))

# 计算各改进因子的贡献
baseline_acc = all_results['baseline']
print(f"\n=== 改进因子贡献分析 ===")
print(f"基线模型准确率: {baseline_acc:.2f}%")
print(f"残差连接提升: {all_results['residual'] - baseline_acc:.2f}%")
print(f"更深更宽网络提升: {all_results['deeper_wider'] - baseline_acc:.2f}%")
print(f"最佳优化器提升: {max(optimizer_results.values()) - baseline_acc:.2f}%")
print(f"最佳数据增强提升: {max(augmentation_results.values()) - baseline_acc:.2f}%")
print(f"最佳注意力机制提升: {max(attention_results.values()) - baseline_acc:.2f}%")


In [ ]:
# 可视化结果
plt.figure(figsize=(15, 10))

# 1. 各方法准确率对比
plt.subplot(2, 2, 1)
plt.bar(range(len(results_df)), results_df['Accuracy'])
plt.xticks(range(len(results_df)), results_df['Method'], rotation=45, ha='right')
plt.ylabel('Test Accuracy (%)')
plt.title('Tiny-ImageNet: All Methods Comparison')
plt.grid(True, alpha=0.3)

# 2. 训练曲线对比
plt.subplot(2, 2, 2)
plt.plot(baseline_history['test_acc'], label='Baseline', linewidth=2)
plt.plot(residual_history['test_acc'], label='Residual', linewidth=2)
plt.plot(deeper_history['test_acc'], label='Deeper/Wider', linewidth=2)
plt.xlabel('Epoch')
plt.ylabel('Test Accuracy (%)')
plt.title('Training Progress Comparison')
plt.legend()
plt.grid(True, alpha=0.3)

# 3. 改进因子贡献
plt.subplot(2, 2, 3)
improvements = {
    'Residual': all_results['residual'] - baseline_acc,
    'Deeper/Wider': all_results['deeper_wider'] - baseline_acc,
    'Best Optimizer': max(optimizer_results.values()) - baseline_acc,
    'Best Augmentation': max(augmentation_results.values()) - baseline_acc,
    'Best Attention': max(attention_results.values()) - baseline_acc
}
plt.bar(improvements.keys(), improvements.values())
plt.ylabel('Accuracy Improvement (%)')
plt.title('Improvement Factor Contributions')
plt.xticks(rotation=45)
plt.grid(True, alpha=0.3)

# 4. 优化器对比
plt.subplot(2, 2, 4)
plt.bar(optimizer_results.keys(), optimizer_results.values())
plt.ylabel('Test Accuracy (%)')
plt.title('Optimizer Comparison')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()


## 5. 最佳组合实验

基于前面的实验结果，组合最佳因子进行最终实验


In [ ]:
print("=== 最佳组合实验 ===")

# 确定最佳配置
best_model_type = 'self_attention'  # 假设自注意力效果最好
best_opt = best_optimizer
best_aug = best_augmentation

print(f"最佳模型: {best_model_type}")
print(f"最佳优化器: {best_opt}")
print(f"最佳数据增强: {best_aug}")

# 创建最佳组合模型
final_model = get_model(best_model_type, num_classes).to(device)

# 配置最佳优化器
opt_config_dict = get_optimizer_config(best_opt)
optimizer = get_optimizer(final_model, best_opt, **opt_config_dict)

# 配置学习率调度器
scheduler = None
if best_opt in ['AdamW', 'SGD']:
    scheduler_config = get_scheduler_config('cosine', config['num_epochs'])
    scheduler = get_scheduler(optimizer, 'cosine', **scheduler_config)

# 使用最佳数据增强
train_loader_final, test_loader_final, _ = get_dataset_loaders(
    config['dataset'], 
    batch_size=config['batch_size'],
    num_workers=config['num_workers'],
    augmentation_type=best_aug
)

# 训练最终模型
final_history = train_model(
    final_model, train_loader_final, test_loader_final,
    config['num_epochs'], optimizer, criterion, device,
    scheduler=scheduler,
    augmentation=best_aug if best_aug in ['mixup', 'cutmix'] else None,
    print_every=config['print_every']
)

final_acc = max(final_history['test_acc'])
print(f"\n最终模型测试准确率: {final_acc:.2f}%")
print(f"相比基线提升: {final_acc - baseline_acc:.2f}%")


## 6. 实验总结

### 主要发现：
1. **残差连接**：有效解决梯度消失问题，提升模型性能
2. **更深更宽网络**：增加模型容量，但需要更多训练时间
3. **优化器改进**：AdamW + 余弦退火调度器效果最佳
4. **数据增强**：MixUp/CutMix等高级增强方法显著提升泛化能力
5. **注意力机制**：SE和自注意力都能有效提升特征表达能力

### 最佳配置：
- 模型：带自注意力的CNN
- 优化器：AdamW + 余弦退火
- 数据增强：MixUp/CutMix
- 预期准确率：>50%

### Tiny-ImageNet vs CIFAR-10：
- Tiny-ImageNet更复杂，需要更强的模型和训练策略
- 图像分辨率更高（64x64 vs 32x32），类别更多（200 vs 10）
- 需要更多的训练时间和更小的batch size
